In [1]:
import pandas as pd
import numpy as np

In [2]:
combined_df=pd.read_csv("Final Dataset with VPN and Non VPN.csv")

In [3]:
row_count, column_count = combined_df.shape
print("Row count:", row_count)
print("Column count:", column_count)


Row count: 623290
Column count: 74


In [4]:
class_distribution = combined_df['Class'].value_counts()
print("\nClass Distribution:")
print(class_distribution)


Class Distribution:
Class
VPN        423385
NON VPN    199905
Name: count, dtype: int64


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D,Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

In [20]:
# Assuming combined_df is your DataFrame containing the data
vpn_packets = combined_df[combined_df['Class'] == 'VPN'].head(10000)

remaining_packets = combined_df[(combined_df['Class'] != 'VPN')].head(10000)

# Concatenate the first 200000 VPN packets with the remaining packets
final_dataset = pd.concat([vpn_packets, remaining_packets], ignore_index=True)
print(final_dataset.shape)



(20000, 74)


In [21]:
unwanted_columns = ['frame.len','tcp.flags.syn','tcp.flags.ack','tcp.flags.push', 'tcp.flags.reset',\
                   'data.len']  # Add all unwanted column names here
# Remove unwanted columns from the DataFrame
filtered_df = final_dataset.drop(columns=unwanted_columns)

# Display the filtered DataFrame
print(filtered_df)


       ip.proto  tcp.srcport  tcp.dstport  udp.srcport  udp.dstport  tcp.seq   
0          17.0          NaN          NaN      15685.0      40002.0      NaN  \
1          17.0          NaN          NaN      40002.0      15685.0      NaN   
2          17.0          NaN          NaN      15685.0      40027.0      NaN   
3          17.0          NaN          NaN      40027.0      15685.0      NaN   
4          17.0          NaN          NaN      23581.0         53.0      NaN   
...         ...          ...          ...          ...          ...      ...   
19995      17.0          NaN          NaN      33306.0      19305.0      NaN   
19996      17.0          NaN          NaN      19305.0      33306.0      NaN   
19997      17.0          NaN          NaN      33306.0      19305.0      NaN   
19998      17.0          NaN          NaN      19305.0      33306.0      NaN   
19999      17.0          NaN          NaN      19305.0      33306.0      NaN   

       frame.time_epoch  tcp.stream  ip

In [22]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit and transform the 'Label' column
filtered_df['Label'] = label_encoder.fit_transform(filtered_df['Label'])
filtered_df['Class'] = label_encoder.fit_transform(filtered_df['Class'])
 # Display the encoded labels
print("Encoded Labels:")
print(filtered_df['Label'].value_counts())
print("Encoded Class:")
print(filtered_df['Class'].value_counts())

Encoded Labels:
Label
3    12739
2     6769
0      319
1      173
Name: count, dtype: int64
Encoded Class:
Class
1    10000
0    10000
Name: count, dtype: int64


In [23]:
# Mapping original labels to encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'NON VPN': 0, 'VPN': 1}


In [24]:
X = filtered_df.drop(columns=['Class'])
y = filtered_df['Class']


In [25]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
num_classes = len(encoder.classes_)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(16000, 67)
(16000,)


In [27]:
from sklearn.impute import SimpleImputer

# Impute missing values in the features
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test= imputer.transform(X_test)

In [28]:
x_training_data = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
print(x_training_data.shape)


(16000, 31, 1)


In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, SimpleRNN, Dropout

# Define input shape based on your data
input_shape = (x_training_data.shape[1], 1)  # Shape of your input data assuming it's sequential

# Define the number of classes for the output
num_classes = 2  # Assuming you have 6 classes for the output

# Create the combined RNN-CNN model
model = Sequential([
    # CNN layers
    Conv1D(32, 3, activation='relu', input_shape=input_shape),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    #Flatten(),

    # RNN layers
    SimpleRNN(units=64, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=64),
    Dropout(0.2),

    # Output layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=24, validation_data=(X_test, y_test))


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 29, 32)            128       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 14, 32)           0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 12, 64)            6208      
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 6, 64)            0         
 1D)                                                             
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 6, 64)             8256      
                                                                 
 dropout_7 (Dropout)         (None, 6, 64)            

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define input shape based on your data
input_shape = (x_training_data.shape[1], 1)  # Shape of your input data

# Define the number of classes for the output
num_classes = 2  # Assuming you have 6 classes for the output

# Create the RNN model
rnn_model = Sequential([
    SimpleRNN(units=64, return_sequences=True, input_shape=input_shape),
    Dropout(0.2),
    SimpleRNN(units=64, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=64),
    Dropout(0.2),
    Dense(units=num_classes, activation='softmax')
])

# Compile the model
rnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Print the model summary
rnn_model.summary()

# Train the model with early stopping
history = rnn_model.fit(X_train, y_train, epochs=10, batch_size=32,
                        validation_data=(X_test, y_test), callbacks=[early_stopping])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_9 (SimpleRNN)    (None, 31, 64)            4224      
                                                                 
 dropout_9 (Dropout)         (None, 31, 64)            0         
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 31, 64)            8256      
                                                                 
 dropout_10 (Dropout)        (None, 31, 64)            0         
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 2)                

In [32]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.0958894563373178e-05
Test accuracy: 1.0


In [33]:
# Evaluate the model
score = rnn_model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.010884657502174377
Test accuracy: 0.9955000281333923
